In [ ]:
#| default_exp expanding

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2

In [ ]:
#| include: false
import random

from nbdev.showdoc import *

In [ ]:
#| export
from math import sqrt
from typing import Callable, Optional

import numpy as np
from numba import njit  # type: ignore

from window_ops.rolling import *

In [ ]:
import pandas as pd

np.random.seed(0)
array = np.random.rand(100)
series = pd.Series(array)

In [ ]:
#| exporti
@njit
def _expanding_op(rolling_op: Callable, x: np.ndarray, min_samples: int = 1) -> np.ndarray:
    n_samples = x.size
    return rolling_op(x, window_size=n_samples, min_samples=min_samples)

In [ ]:
#| export
@njit
def expanding_mean(input_array: np.ndarray) -> np.ndarray:
    return _expanding_op(rolling_mean, input_array)

In [ ]:
np.testing.assert_allclose(expanding_mean(array), series.expanding().mean().values)

In [ ]:
#| export
@njit
def expanding_std(input_array: np.ndarray) -> np.ndarray:
    return _expanding_op(rolling_std, input_array, min_samples = 2)

In [ ]:
np.testing.assert_allclose(expanding_std(array), series.expanding().std().values)

In [ ]:
#| export
@njit
def expanding_max(x: np.ndarray) -> np.ndarray:
    return _expanding_op(rolling_max, x)

In [ ]:
np.testing.assert_allclose(expanding_max(array), series.expanding().max())

In [ ]:
#| export
@njit
def expanding_min(x: np.ndarray) -> np.ndarray:
    return _expanding_op(rolling_min, x)

In [ ]:
np.testing.assert_allclose(expanding_min(array), series.expanding().min())

## Seasonal

In [ ]:
seasons = np.arange(array.size) % 7
grouped_series = series.groupby(seasons)

In [ ]:
#| exporti
@njit
def _seasonal_expanding_op(rolling_op: Callable, x: np.ndarray, season_length: int,
                           min_samples: int = 1) -> np.ndarray:
    n_samples = x.size
    return rolling_op(x, season_length=season_length, window_size=n_samples, min_samples=min_samples)

In [ ]:
#| export
@njit
def seasonal_expanding_mean(x: np.ndarray, season_length: int) -> np.ndarray:
    return _seasonal_expanding_op(seasonal_rolling_mean, x, season_length)

In [ ]:
np.testing.assert_allclose(
    seasonal_expanding_mean(array, 7),
    grouped_series.transform(lambda y: y.expanding().mean())
)

In [ ]:
#| export
@njit
def seasonal_expanding_std(x: np.ndarray, season_length: int) -> np.ndarray:
    return _seasonal_expanding_op(seasonal_rolling_std, x, season_length, min_samples=2)

In [ ]:
np.testing.assert_allclose(
    seasonal_expanding_std(array, 7),
    grouped_series.transform(lambda y: y.expanding().std())
)

In [ ]:
#| export
@njit
def seasonal_expanding_min(x: np.ndarray, season_length: int):
    return _seasonal_expanding_op(seasonal_rolling_min, x, season_length)

In [ ]:
np.testing.assert_allclose(
    seasonal_expanding_min(array, 7),
    grouped_series.transform(lambda y: y.expanding().min())
)

In [ ]:
#| export
@njit
def seasonal_expanding_max(x: np.ndarray, season_length: int):
    return _seasonal_expanding_op(seasonal_rolling_max, x, season_length)

In [ ]:
np.testing.assert_allclose(
    seasonal_expanding_min(array, 7),
    grouped_series.transform(lambda y: y.expanding().min())
)